In [15]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import os.path as op
import shutil
from zipfile import ZipFile
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from utils import get_image_paths, word_from_image_path, preprocess_image, print_im, TextTransform, N_CHARS, SEQUENCE_LENGTH, IMAGE_DIMENSIONS
from utils import NgramTransform, load_trained_CNN_weights
from multi_gpu import make_parallel

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Building N-grams Neural Network

In [16]:
cnn = load_trained_CNN_weights('p_model_0.h5')

In [17]:
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Reshape, Activation, GlobalAveragePooling2D
from keras.layers import Flatten

## Decapitation of previous trained CNN
input = cnn.layers[0].input
output = cnn.layers[-4].output
base_ngrams = Model(input=input, output=output)
# input_shape = (None, 32, 100, 1,)

ngrams = Sequential()
ngrams.add(base_ngrams)

ngrams.add(Dense(10000, activation='sigmoid', name= "10kdense")) 


ngrams.compile(optimizer='adam', loss='categorical_crossentropy')

In [18]:
# ## Parellelize model 
# p_model_ngrams = make_parallel(ngrams, 4)
# p_model_ngrams.compile(optimizer='adam', loss='mse')

### Loading Images and Batch

In [19]:
from tqdm import tqdm, tqdm_notebook
base_dir = '/mnt/c/Users/benoua/Google Drive/Master X - Data Sciences/2_Structured Data/Structured-Data/mjsynth'
# base_dir = '/mnt/mjsynth/mnt/ramdisk/max/90kDICT32px/'

images_paths = get_image_paths(base_dir)
#%%time
ims = []
errors_1, errors_2 = [], []
for i, im in enumerate(images_paths[:12]):
    try:
        ims.append(preprocess_image(img_to_array(load_img(im, grayscale=True))))
    except OSError:
        errors_1.append(i)

In [20]:
batch = np.stack(ims).reshape(12, 32, 100, 1)
batch.shape

(12, 32, 100, 1)

In [51]:
### NGram model output vector 
ngram_transformer = NgramTransform()

In [52]:
batch_y = ngram_transformer.make_batch_labels(images_paths[:12], False)
batch_y = batch_y.reshape(batch_y.shape[0], batch_y.shape[2]).astype(float)

### Scaling gradients/classes to labels frequencies

In [65]:
# check which are the most frequent ngrams
ngram_transformer.ngram_from_matrix(ngram_transformer.X_tf > 5000000)

['a', 'e', 'i', 'n', 'r', 's']

In [66]:
# Creating labels dictionnary
labels_dict = {i: ngram_transformer.X_tf[i] for i in range(10000) }

In [68]:
# fitting with respect to the labels scaling
# ngrams.fit(batch, batch_y, nb_epoch=1, class_weight=labels_dict)

### Predictions

In [ ]:
predictions = ngrams.predict(batch)